### Read Smallcases

In [22]:
import numpy as np
import pandas as pd

In [25]:
ids = [
    {"id": "SCAW_0001", "name": "All Weather Investing"},
    {"id": "SCMO_0015", "name": "Safe Haven"},
    {"id": "SCMO_0026", "name": "Growth & Income"},
    {"id": "SCMO_0016", "name": "The Naked Trader"},
    {"id": "SCMO_0024", "name": "The PE List"},
    {"id": "SCMO_0013", "name": "Dividend Stars"},
    {"id": "SCMO_0014", "name": "Dividend Aristocrats"},
    {"id": "SCNM_0025", "name": "Electric Mobility"},
    {"id": "SCNM_0012", "name": "Rising Rural Demand"},
    {"id": "SCTR_0014", "name": "House of Tata"},
    {"id": "SCTR_0017", "name": "House of Murugappa"},
    {"id": "SCTR_0010", "name": "Realty Tracker"},
]
id = ids[0]["id"]
smallcase = pd.ExcelFile(f"data/smallcases/{id}_Timeline.xlsx")

constituents = pd.read_excel(smallcase, sheet_name="Historical Constituents")
constituents.head()

,Date Range,Constituents,Weightage
0,2018-07-16 to 2018-07-16,Nippon India ETF Nifty 50 BeES,0.19
1,NaN,Nippon India ETF Nifty Next 50 Junior BeES,0.14
2,NaN,Nippon India ETF Gold BeES,0.41
3,NaN,Nippon India ETF Nifty 1D Rate Liquid BeES,0.25
4,2018-07-17 to 2018-09-27,Nippon India ETF Nifty 50 BeES,0.24


In [26]:
indexes = pd.read_excel(smallcase, sheet_name="Historical Index Values")
indexes.head()

,Date,All Weather Investing,NIFTY 100,Rebalance Occured
0,2018-07-17,100.00,100.00,True
1,2018-07-18,99.33,99.63,NaN
2,2018-07-19,99.29,99.37,NaN
3,2018-07-20,99.52,99.87,NaN
4,2018-07-23,100.22,100.68,NaN


### Map and POST Smallcase Name to Upstox Instrument

In [27]:
# Connect to db
import os
import sys

backend_path = os.path.abspath(os.path.join(os.getcwd(), '../backend'))
sys.path.append(backend_path)
from app.internal.firebase import db

In [28]:
postable_constituents = constituents["Constituents"].unique()
len(postable_constituents)

5

In [29]:
successful_constitents = []
failed_constitents = []

async def query_ticker_name(name: str):
    return await db.collection("tickers").where("name", ">=", name).where("name", "<=", name + '\uf8ff').get()

for c in postable_constituents:
    q = c.upper()
    docs = await query_ticker_name(q)
    if len(docs) == 0:
        failed_constitents.append(c)
    else:
        for e in docs:
            await db.collection("tickers").document(e.id).update({
                "smallcase_name": c
            })
            successful_constitents.append((c, e.id))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:302: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
/var/folders/4x/3kzzlsjs1q36v35r81960pt40000gn/T/ipykernel_36407/2676511566.py:5: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return await db.collection("tickers").where("name", ">=", name).where("name", "<=", name + '\uf8ff').get()


In [30]:
len(successful_constitents), len(failed_constitents)

(0, 5)

In [31]:
# # All return upper

def replace_ltd(s):
    return s.upper().replace("LTD", "LIMITED")

# def replace_technologies(s):
#     return s.upper().replace("TECHNOLOGIES", "TECHNO.")

# def replace_company(s):
#     return s.upper().replace("COMPANY", "CO")

# def replace_corporation(s):
#     return s.upper().replace("CORPORATION", "CORP")

# Ltd -> Limited
for f in failed_constitents:
    q = replace_ltd(f)
    docs = await query_ticker_name(q)
    if len(docs) != 0:
        for e in docs:
            await db.collection("tickers").document(e.id).update({
                "smallcase_name": f
            })
            successful_constitents.append((f, e.id))
            failed_constitents.remove(f)

/var/folders/4x/3kzzlsjs1q36v35r81960pt40000gn/T/ipykernel_36407/2676511566.py:5: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return await db.collection("tickers").where("name", ">=", name).where("name", "<=", name + '\uf8ff').get()


In [32]:
len(successful_constitents), len(failed_constitents)

(0, 5)

In [33]:
nse = pd.read_csv("data/upstox/NSE.csv")
equities = nse.query('instrument_type == "EQUITY" and exchange == "NSE_EQ"')
equities.head()

,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange
77,NSE_EQ|DUMMYSAN005,14747.0,011NSETEST,011NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
78,NSE_EQ|DUMMYSAN006,14751.0,021NSETEST,021NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
79,NSE_EQ|DUMMYSAN007,14753.0,031NSETEST,031NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
80,NSE_EQ|DUMMYSAN008,14755.0,041NSETEST,041NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
81,NSE_EQ|DUMMYSAN009,14758.0,051NSETEST,051NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ


In [34]:
import difflib
df = equities.filter(['name', "exchange_token"])
tickers = pd.DataFrame() 
r = {}
for f in failed_constitents:
    df['similarity'] = df["name"].apply(lambda x: difflib.SequenceMatcher(None, x.lower(), f.lower()).ratio())
        
    top_match = df.sort_values(by='similarity', ascending=False).head(1)
    top_match['smallcase_name'] = f
        
    tickers = pd.concat([tickers, top_match], ignore_index=True)

tickers

,name,exchange_token,similarity,smallcase_name
0,NIP IND ETF NIFTY BEES,10576.0,0.846154,Nippon India ETF Nifty 50 BeES
1,NIP IND ETF JUNIOR BEES,10939.0,0.707692,Nippon India ETF Nifty Next 50 Junior BeES
2,NIP IND ETF GOLD BEES,14428.0,0.893617,Nippon India ETF Gold BeES
3,NIP IND ETF LIQUID BEES,11006.0,0.707692,Nippon India ETF Nifty 1D Rate Liquid BeES
4,ZERODHAAMC - LIQUIDCASE,21750.0,0.581818,Zerodha Nifty 1D Rate Liquid ETF


In [35]:
payload = tickers[['exchange_token', 'smallcase_name']].to_dict(orient='records')
for p in payload:
    id = str(int(p['exchange_token']))
    await db.document(f"tickers/{id}").update({"smallcase_name": p["smallcase_name"]})
    successful_constitents.append((p["smallcase_name"], id))

In [36]:
len(successful_constitents), len(successful_constitents) == len(postable_constituents)

(5, True)